In [1]:
from pathlib import Path

import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
base_path = Path(".").absolute().parent

In [3]:
class GestureDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.X = []
        self.y = []
        self.initialize()
    
    def initialize(self):
        gestures = list(sorted((Path(".").absolute().parent / "data").iterdir()))
        for y, gesture in enumerate(gestures):
            for file in gesture.glob("*.npy"):
                self.X.append(file)
                self.y.append(y)
                
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        X = np.load(str(self.X[idx])).astype(np.float32)
        X = torch.from_numpy(X)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        
        return X, y
                

In [4]:
ds = GestureDataset()

In [5]:
train_split, test_split = random_split(ds, [0.7, 0.3], generator=torch.Generator().manual_seed(42))

In [6]:
train_dataloader = DataLoader(train_split, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_split, batch_size=16, shuffle=True)

In [7]:
X,y = next(iter(train_dataloader))

In [8]:
class GestureClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        
        self.linear_stack1 = nn.Sequential(
            nn.Linear(in_features=42, out_features=20),
            nn.ReLU(),
            nn.Dropout(.2)
        )
        
        self.linear_stack2 = nn.Sequential(
            nn.Linear(in_features=20, out_features=10),
            nn.ReLU(),
            nn.Dropout(.4)
        )
        
        self.output = nn.Linear(in_features=10, out_features=n_classes)
        
    def forward(self, x):
        x = self.linear_stack1(x)
        x = self.linear_stack2(x)
        x = self.output(x)
        return x

In [12]:
class Trainer:
    def __init__(self, model, train_dl, test_dl, epochs=1000, patience=10):
        self.model = model
        self.train_dl = train_dl
        self.test_dl = test_dl
        self.epochs = epochs
        self.patience = 10
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimzer = torch.optim.Adam(model.parameters())
        
        self.best_acc = 0
        self.current_patience = patience
        self.best_model = None
        
    def train(self):
        for t in range(self.epochs):
            print(f"Epoch: {t+1}")
            self.train_loop()
            self.test_loop()
            if self.current_patience == 0:
                self.model.state_dict = self.best_model
                print("Patience broken")
                break
        print("Done")
        torch.save(self.model.state_dict(),  str(base_path / "models" / "gesture_clf.pth"))
        
    def train_loop(self):
        size = len(self.train_dl.dataset)
        for batch, (X, y) in enumerate(self.train_dl):
            pred = self.model(X)
            loss = self.loss_fn(pred, y)
            
            self.optimzer.zero_grad()
            loss.backward()
            self.optimzer.step()
            
            if batch % 10== 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss} [{current}/{size}]")
                
    def test_loop(self):
        num_batches = len(self.test_dl)
        test_loss, correct = 0, 0
        
        with torch.no_grad():
            for X, y in self.test_dl:
                pred = self.model(X)
                test_loss += self.loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item() / self.test_dl.batch_size
                
        test_loss /= num_batches
        correct /= num_batches
        print(f"Test Acc: {correct * 100}%, Avg loss: {test_loss}")
        
        if correct > self.best_acc:
            self.best_acc = correct
            self.best_model = self.model.state_dict
            self.current_patience = self.patience
        else:
            self.current_patience -= 1
            

In [13]:
model = GestureClassifier(3)
trainer = Trainer(model, train_dataloader, test_dataloader, epochs=100)

In [14]:
trainer.train()

Epoch: 1
loss: 1.1184861660003662 [0/1567]
loss: 1.0942144393920898 [320/1567]
loss: 1.084151029586792 [640/1567]
loss: 1.0800232887268066 [960/1567]
loss: 1.0501691102981567 [1280/1567]
Test Acc: 45.535714285714285%, Avg loss: 1.0690373125530424
Epoch: 2
loss: 1.0976680517196655 [0/1567]
loss: 1.0522868633270264 [320/1567]
loss: 1.0519143342971802 [640/1567]
loss: 1.0148811340332031 [960/1567]
loss: 0.9660263061523438 [1280/1567]
Test Acc: 49.25595238095239%, Avg loss: 1.041172879082816
Epoch: 3
loss: 1.0097383260726929 [0/1567]
loss: 0.9801128506660461 [320/1567]
loss: 1.0211849212646484 [640/1567]
loss: 1.0758389234542847 [960/1567]
loss: 0.9867303967475891 [1280/1567]
Test Acc: 56.25%, Avg loss: 0.9822352088633037
Epoch: 4
loss: 1.0370632410049438 [0/1567]
loss: 0.9014425873756409 [320/1567]
loss: 0.991741955280304 [640/1567]
loss: 0.9720467925071716 [960/1567]
loss: 0.8719087243080139 [1280/1567]
Test Acc: 61.16071428571429%, Avg loss: 0.8994372515451341
Epoch: 5
loss: 0.927555441

In [59]:
x = torch.randn(2, 42)

In [61]:
x.size()

torch.Size([2, 42])

In [72]:
class ConvClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, 3, padding=1),
            nn.ReLU()
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 64, 3, padding=1),
            nn.ReLU()
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(2688, 1024),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        print(x.size())
#         x = torch.flatten(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
        
        return x

In [73]:
m = ConvClassifier()

In [70]:
x = torch.randn(16, 1, 42)

In [71]:
x.size()

torch.Size([16, 1, 42])

In [74]:
out = m(x)

torch.Size([16, 64, 42])


In [53]:
out

tensor([-0.0376,  0.0040, -0.0273,  0.0151, -0.0162,  0.0375,  0.0184,  0.0556,
        -0.0143, -0.0587], grad_fn=<AddBackward0>)

In [54]:
loss = nn.CrossEntropyLoss()

In [55]:
loss(out, torch.randn(10))

tensor(-4.2438, grad_fn=<DivBackward1>)

In [107]:
t1 = torch.randn(1, 10)
t2 = torch.randn(1, 10)
t3 = torch.stack((t1, t2))

In [113]:
t3.size()

torch.Size([2, 1, 10])

In [130]:
flatten = nn.Flatten()

In [132]:
flatten(t3).size()

torch.Size([2, 10])

In [129]:
t1

tensor([[ 0.3892, -0.8036, -2.2099, -0.6902,  1.9779,  2.4583, -0.4430, -1.1128,
          0.7266,  1.5586]])

In [133]:
x = np.load(base_path / "data" / "point_up" / "00000.npy")

In [134]:
x

array([0.46355143, 0.98830217, 0.50423163, 0.97798246, 0.53948539,
       0.90498883, 0.54393858, 0.83076483, 0.5424158 , 0.76208138,
       0.51942861, 0.78046501, 0.52059108, 0.69570863, 0.51853007,
       0.62922424, 0.5154928 , 0.57830215, 0.47013804, 0.76820189,
       0.47058395, 0.72023898, 0.48166522, 0.7768656 , 0.4931978 ,
       0.82481194, 0.42324179, 0.78242403, 0.43131328, 0.80025256,
       0.4477661 , 0.88284487, 0.46112165, 0.9374404 , 0.37857261,
       0.81729281, 0.39336514, 0.85285443, 0.41415969, 0.91794497,
       0.43098775, 0.9552092 ])